## Introduction to images

In [2]:
using Images
using FileIO
using TestImages

In [4]:
using TestImages, Images, ImageView
#Pkg.add("ImageView")

## Images are just arrays

For most purposes, any AbstractArray can be treated as an image. For example,

In [5]:
using Images

img = rand(640,480)               # a random Float64 image
img = rand(RGB{N0f8}, 256, 256)   # a random RGB image, 8 bits per channel

# select a region-of-interest from a larger image
imgc = img[200:245, 17:42]        # makes a copy
imgv = @view img[200:245, 17:42]  # makes a view

# an image that starts black in the upper left and gets bright in the lower right:
img = reshape(linspace(0,1,10^4), 100, 100)

# a 3d box image
img = zeros(128, 128, 80)
img[20:100, 20:100, 10:70] = 1


1

Some add-on packages enable additional behavior. For example,



In [11]:
using Images, Unitful, AxisArrays
using Unitful: mm, s

img = AxisArray(rand(256, 256, 100, 50),
                (:x, :y, :z, :time),
                (0.4mm, 0.4mm, 1mm, 2s))


4-dimensional AxisArray{Float64,4,...} with axes:
    :x, 0.0 mm:0.4 mm:102.0 mm
    :y, 0.0 mm:0.4 mm:102.0 mm
    :z, 0 mm:1 mm:99 mm
    :time, 0 s:2 s:98 s
And data, a 256×256×100×50 Array{Float64,4}:
[:, :, 1, 1] =
 0.921288   0.290008   0.106776  0.721001   …  0.719048   0.226941  0.992768
 0.824954   0.507762   0.510644  0.527419      0.0660953  0.152058  0.420484
 0.490485   0.969566   0.524972  0.340051      0.664532   0.852185  0.419298
 0.664166   0.265713   0.820797  0.89535       0.639367   0.928768  0.60548 
 0.0535329  0.600981   0.823596  0.697325      0.352661   0.521062  0.946371
 0.967457   0.994132   0.190564  0.819922   …  0.269197   0.252055  0.764286
 0.538756   0.617016   0.629664  0.926192      0.740424   0.349834  0.571643
 0.139646   0.614745   0.71296   0.579533      0.426548   0.453981  0.384542
 0.23184    0.452528   0.786508  0.543687      0.229721   0.542047  0.879292
 0.820737   0.552833   0.794924  0.798632      0.240725   0.607757  0.928023
 0.95396  

defines a 4d image (3 space dimensions plus one time dimension) with the specified name and physical pixel spacing for each coordinate. The AxisArrays package supports rich and efficient operations on such arrays, and can be useful to keep track of not just pixel spacing but the orientation convention used for multidimensional images.

JuliaImages interoperates smoothly with AxisArrays and many other packages. As further examples,


- the `ImageMetadata` package (incorporated into Images itself) allows you to "tag" images with custom metadata


- the `IndirectArrays` package supports indexed (colormap) images


- the `MappedArrays` package allows you to represent lazy value-transformations, facilitating work with images that may be too large to store in memory at oncthe MappedArrays package allows you to represent lazy value-transformations, facilitating work with images that may be too large to store in memory at once


- `ImageTransformations` allows you to encode rotations, shears, deformations, etc., either eagerly or lazilImageTransformations allows you to encode rotations, shears, deformations, etc., either eagerly or lazil-y

It is very easy to define new array types in Julia–and consequently specialized images or operations–and have them interoperate smoothly with the vast majority of functions in JuliaImages.



## Colors, the 0-to-1 intensity scale, and views


In JuliaImages, by default all images are displayed assuming that 0 means "black" and 1 means "white" or "saturated" (the latter applying to channels of an RGB image). Perhaps surprisingly, this 0-to-1 convention applies even when the intensities are encoded using only 8-bits per color channel. JuliaImages uses a special type, N0f8, that interprets an 8-bit "integer" as if it had been scaled by 1/255, thus encoding values from 0 to 1 in 256 steps. N0f8 numbers (standing for Normalized, with 0 integer bits and 8 fractional bits) obey standard mathematical rules, and can be added, multiplied, etc. There are types like N0f16 for working with 16-bit images (and even N2f14 for images acquired with a 14-bit camera, etc.).

This infrastructure allows us to unify "integer" and floating-point images, and avoids the need for special conversion functions that change the value of pixels when your main goal is simply to change the type (numeric precision and properties) used to represent the pixel.

Because images are just arrays, some environments (e.g., IJulia/Jupyter) will display numeric arrays as arrays (using a text representation) but will display 2d arrays that have Colorant elements as images. You can "convert" in the following ways:



In [13]:
img = colorview(Gray, rand(8, 8))          # encodes as Gray{Float64}, so displays as image
img = colorview(RGB, rand(3, 8, 8))        # encodes as a 2d RGB{Float64} array
img = colorview(RGB, rand(N0f8, 3, 8, 8))  # uses only 8 bits per channel

# The following two "convert" between representation as an 8-bit RGB
# image and as a 3×m×n UInt8 array
img = colorview(RGB, normedview(A))
A = rawview(channelview(rand(RGB{N0f8}, 8, 8)))


LoadError: [91mUndefVarError: A not defined[39m

All of these "conversions" actually create views, meaning that no copies of the underlying storage are made unless you call copy on the result.



## Load image

In [ ]:
img = testimage("mandrill")
imshow(img)